In [2]:
import torch 
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from model import*
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision import transforms
from torch.autograd import Variable

In [3]:
learning_rate = 1e-3
batch_size = 64
epoches = 50

In [5]:
trans_img = transforms.ToTensor()

trainset = MNIST('./data', train=True, transform=trans_img, download=True)
testset = MNIST('./data', train=False, transform=trans_img, download=True)

trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=4)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=4)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:06<00:00, 1610971.52it/s]


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 136238.71it/s]


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:01<00:00, 1138727.62it/s]


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 2270893.88it/s]

Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw



In [7]:
lenet = LeNet()
lenet = lenet.cuda()

In [8]:
lenet 

LeNet(
  (net): Sequential(
    (0): ConvLayers(
      (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
      (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
      (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (1): FCNN(
      (fc1): Linear(in_features=400, out_features=120, bias=True)
      (fc2): Linear(in_features=120, out_features=84, bias=True)
      (fc3): Linear(in_features=84, out_features=10, bias=True)
    )
  )
)

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(lenet.parameters(), lr=learning_rate)

In [10]:
from tqdm.notebook import tqdm

In [18]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint

In [15]:
class LeNetTrain(pl.LightningModule):
    def __init__(self):
        super(LeNetTrain, self).__init__()
        self.lenet = LeNet()
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.lenet.parameters(), lr=learning_rate)
        
    def forward(self, x):
        return self.lenet(x)
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        x, y = x.cuda(), y.cuda()
        y_hat = self.lenet(x)
        loss = self.criterion(y_hat, y)
        self.log('train_loss', loss)
        return loss
    
    def configure_optimizers(self):
        return self.optimizer
    
    def train_dataloader(self):
        return trainloader

In [16]:
lenet_train = LeNetTrain()

In [17]:
lenet_train

LeNetTrain(
  (lenet): LeNet(
    (net): Sequential(
      (0): ConvLayers(
        (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
        (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
        (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
      (1): FCNN(
        (fc1): Linear(in_features=400, out_features=120, bias=True)
        (fc2): Linear(in_features=120, out_features=84, bias=True)
        (fc3): Linear(in_features=84, out_features=10, bias=True)
      )
    )
  )
  (criterion): CrossEntropyLoss()
)

In [19]:
trainer = pl.Trainer(max_epochs=10,accelerator='gpu',devices=1,callbacks=[ModelCheckpoint()])

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [20]:
trainer.fit(lenet_train, trainloader)

You are using a CUDA device ('NVIDIA GeForce RTX 3080') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Missing logger folder: d:\portfolio\paper_implementations\LeNet\lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params
-----------------------------------------------
0 | lenet     | LeNet            | 61.7 K
1 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
61.7 K    Trainable params
0         Non-trainable params
61.7 K    Total params
0.247     Total estimated model params size (MB)
c:\Users\BiswanathSaha\miniconda3\envs\stable\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:436: Consider setting `persistent_workers=T

Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.
